In [1]:
#### Dependencies
import numpy as np
import time

In [2]:
## To load data
# h : low profit or high profit
# size : selected size of data
def load_data(h,size):
    if h == 1:
        data = np.loadtxt('h'+str(size)+'.txt')
    elif h == 0:
        data = np.loadtxt('l'+str(size)+'.txt')
    else:
        print('Error.')
    proftis = data[:,3]
    data = data[:,1:3]
    return data,proftis

In [3]:
## Data variable
data = None
### Profits variable
profits = None
## Distance matrix variable not to calculate each time
disMat = None
### iteration number of inner SA
inner_epochs = 1000
### iteration number of meta SA
meta_epochs = 5000
## Size of data
size = None

In [4]:
## modication of tour 
## if wanted insertion and deletion method can be change here
## We use random insertion, so it does not do anything.
def modification(tour):
    return tour
#### Gives initial tour
## size : size of tour
def initial_tour(size):
    ### l : intitial size
    ### it is arrange according to problem(high or low)
    l = int(size*0.5)
    tour = np.zeros((l+1))
    tour[0] = 0
    tour[l] = 0
    tour[1:l] = np.random.choice(size-1,l-1,replace=False)+1
    return tour
### Calculation tour cost
## we convert problem to minimization so profits are minus, costs are plus
## tour : permutation representation tour
def tour_cost(tour):
    global disMat,profits
    tour_length = len(tour)
    cost = 0
    for i in range(tour_length-1):
        cost += disMat[int(tour[i]),int(tour[i+1])]
        cost -= profits[i]
    return cost
## inner simulating annealing piece
## tour : initiak tour for sa
## epoch : number of iterations
## cost  : initial cost
def simulated_annealing(tour,epochs,cost):
    length = len(tour)
    ### initial and final temperature
    Tinit = 10
    Tfinal = 1
    for k in range(epochs):
        ### temperature annealing regime
        T = np.power(Tinit*(Tfinal/Tinit),k/epochs)
        #### two exchange method
        choices = np.random.choice(length-2,2,replace=False);
        choices = choices +1
        i = min(choices)
        j = max(choices)
        new_tour = np.copy(tour);
        new_tour[i:j+1] = tour[i:j+1][::-1];
        new_cost = tour_cost(new_tour)
        delta = new_cost - cost
        #### calculating change probability
        p = min(np.exp(-delta/T),1)
        ### proposal accepting or rejecting
        if np.random.rand() < p:
            tour = np.copy(new_tour)
            cost = new_cost
    return np.copy(tour),cost
## insert method for meta sa
## tour : initiak tour
## insert a element that is not included
def insert(tour):
    length = len(tour)
    if length == size+1:
        return simulated_annealing(tour,inner_epochs,tour_cost(tour))
    else:
        ### to find exclueded elemetns
        j = np.random.choice(np.setdiff1d(range(1,size),tour),1)
        new_tour = np.zeros(length+1)
        shuffle = np.copy(tour[1:length-1])
        ### random choice
        j_ = np.random.randint(len(shuffle))
        shuffle = np.insert(shuffle,j_,j)
        new_tour[1:length] = modification(shuffle)
        new_cost = tour_cost(new_tour)
        ### new proposal are optimized
        return simulated_annealing(new_tour,inner_epochs,new_cost)
## remove methdod for meta sa
## tour : initial tour
## randomly deleting an element
def remove(tour):
    length = len(tour)
    ### 
    j = np.random.choice(length-2,1)+1
    new_tour = np.delete(tour,j)
    new_tour[1:length-2] = modification(new_tour[1:length-2])
    new_cost = tour_cost(new_tour)
    ### optimization
    return simulated_annealing(new_tour,inner_epochs,new_cost)
### exchange one element with excluded one
### insert + delete used for simplicity
## tour: initial tour
def exchange(tour):
    length = len(tour)
    if length == size+1:
        return simulated_annealing(tour,500,tour_cost(tour))
    else:
        j = np.random.choice(np.setdiff1d(range(1,size),tour),1)
        new_tour = np.zeros(length)
        shuffle = np.copy(tour[1:length-1])
        j_ = np.random.randint(len(shuffle))
        shuffle = np.insert(shuffle,j_,j)
        j_ = np.random.randint(len(shuffle))
        shuffle = np.delete(shuffle,j_)
        new_tour[1:length-1] = modification(shuffle)
        new_cost = tour_cost(new_tour)
        ## proposal optimized
        return simulated_annealing(new_tour,inner_epochs,new_cost)

In [5]:
## the general algoritjm
## s : size of the 
## h : high or low
def meta_sa(s,h):
    global profits,data,disMat,size
    size = s
    data,profits = load_data(h,size)
    disMat = np.zeros((size,size))
    best_tour = None
    best_cost = 0
    for i in range(size):
        for j in range(size):
            disMat[i,j] = np.linalg.norm(data[i,:] - data[j,:])
    ### intial solution
    tour = initial_tour(size)
    cost = tour_cost(tour)
    tour,cost = simulated_annealing(tour,inner_epochs,cost)
    length = len(tour)
    print(cost,length)
    ### initial and final temperature
    Tinit = 5
    Tfinal = 1
    for k in range(1,meta_epochs):
        if k % 100 == 0:
            print(k,cost,len(tour))
        ## temperature rejime
        T = np.power(Tinit*(Tfinal/Tinit),k/meta_epochs)
        p = np.random.rand() 
        ### randomly selecting a move operator
        if p > 0.75:
            ### remove : diversification
            new_tour,new_cost = remove(tour)
        elif 0.75 > p and 0.5 < p:
            ### insert diversification
            new_tour,new_cost = insert(tour)
        elif 0.5 > p and 0.25 < p:
            ### exhanging element
            ## diversification and intensification in terms of size
            new_tour,new_cost = exchange(tour)
        else:
            ## optimizing current solution with starting high temperature
            ### intensification
            new_tour,new_cost = simulated_annealing(tour,inner_epochs,cost)
        delta = new_cost - cost
        p = min(np.exp(-delta/T),1)
        if new_cost < best_cost:
            best_tour = new_tour
            best_cost = new_cost
        if cost <  best_cost:
            best_tour = tour
            best_cost = cost
        if np.random.rand() < p:
            tour = np.copy(new_tour)
            cost = new_cost
    return best_tour

In [ ]:
start = time.time()
tour_1 = meta_sa(51)
cpu_usage = (time.time()-start)
cost_1 = tour_cost(tour_1)
print(cost_1,len(tour_1),cpu_usage)
print(tour_1)

In [ ]:
start = time.time()
tour_2 = meta_sa(76)
cpu_usage = (time.time()-start)
cost_2 = tour_cost(tour_2)
print(cost_2,len(tour_2),cpu_usage)
print(tour_2)

In [ ]:
start = time.time()
tour_3 = meta_sa(101)
cpu_usage = (time.time()-start)
cost_3 = tour_cost(tour_3)
print(cost_3,len(tour_3),cpu_usage)
print(tour_3)